In [1]:
import igraph
import random
import numpy
import pandas

In [2]:
def make_graph(weighted_edges):
    edges = [edge[0:2] for edge in weighted_edges]
    weights = [edge[2] for edge in weighted_edges]

    return igraph.Graph(
        edges=edges,
        edge_attrs={'weight': weights}
    )

def read_graph_from_file(path):
    with open(path) as file:
        lines = file.readlines()
        weighted_edges = []
        for line in lines:
            strings = line.rstrip().split(' ')
            values = [int(s) for s in strings]
            weighted_edges.append(values)

        return make_graph(weighted_edges)

In [84]:
def _fitness(line_graph: igraph.Graph, solution, solution_index):
    total_cost = 0
    for index in range(len(solution)):
        begin_vertex = int(solution[index])
        end_vertex = int(solution[(index + 1) % len(solution)])
        # print(begin_vertex, end_vertex, solution)
        shortest_vertex_path = line_graph.get_shortest_paths(v=begin_vertex, to=end_vertex)[0]

        path_cost = 0
        for vertex in shortest_vertex_path[1:]:
            path_cost += line_graph.vs[vertex]["weight"]

        total_cost += path_cost

    return -total_cost

In [4]:
graph = read_graph_from_file("./graphs/graph_small.dat")

In [5]:
def generate_initial_population(edge_count: int, population_size: int):
    population = [list(range(edge_count)) for _ in range(population_size)]
    for solution in population:
        random.shuffle(solution)
    return population


In [9]:
population = list(map(lambda x: int(x), [9.0, 5.0, 4.0, 1.0, 0.0, 2.0, 22.0, 25.0, 29.0, 6.0, 3.0, 11.0, 14.0, 12.0, 13.0, 15.0, 17.0, 18.0, 24.0, 28.0, 27.0, 20.0, 19.0, 21.0, 23.0, 26.0, 8.0, 7.0, 16.0, 10.0]))

In [18]:
line_graph: igraph.Graph = graph.linegraph()
for vertex in range(line_graph.vcount()):
    line_graph.vs[vertex]["weight"] = graph.es[vertex]["weight"]

_fitness(line_graph, population, 0)

InternalError: Error at src\_igraph\attributes.c:1700: No such attribute. -- Invalid value

In [ ]:
cost = 0
for vertex in [23, 21, 19, 18, 15, 14]:
    cost += line_graph.vs[vertex]["weight"]
cost

In [ ]:
edges, score, index = (numpy.array([23., 22., 24., 27.,  2.,  0.,  3.,  6., 28., 25.,  8.,  9.,  1.,
       29., 18., 20., 13., 12., 16., 17., 26., 19., 15., 14., 11., 10.,
        4.,  5.,  7., 21.]), -281, 0)

In [24]:
def calculate_cost(graph: igraph.Graph, vertices: list[int]):
    cost = 0
    for index in range(len(vertices) - 1):
        start_vertex = vertices[index]
        end_vertex = vertices[index + 1]
        cost += graph.es.find(_source=start_vertex, _target=end_vertex)['weight']

    return cost

In [19]:
def create_vertex_path(graph: igraph.Graph, edges: list[float]):
    edge_dataframe = graph.get_edge_dataframe()
    edge_dataframe.loc[len(edge_dataframe.index)] = edge_dataframe.iloc[0]

    vertex_path = []
    for edge in edges:
        source = edge_dataframe.iloc[int(edge)].source
        target = edge_dataframe.iloc[int(edge)].target

        if len(vertex_path) == 0:
            vertex_path.append(source)
            vertex_path.append(target)
        elif edge == edges[-1]:
            previous_vertex = vertex_path[-1]
            path = graph.get_shortest_paths(v=previous_vertex, to=vertex_path[0], weights="weight")[0]
            for vertex in path[1:]:
                vertex_path.append(vertex)
        elif len(vertex_path) > 0:
            previous_vertex = vertex_path[-1]

            if source == previous_vertex:
                vertex_path.append(target)
            elif target == previous_vertex:
                vertex_path.append(source)
            else:
                source_path = graph.get_shortest_paths(v=previous_vertex, to=source, weights="weight")[0]
                target_path = graph.get_shortest_paths(v=previous_vertex, to=target, weights="weight")[0]

                if source in source_path and target in source_path:
                    for vertex in source_path[1:]:
                        vertex_path.append(vertex)
                elif source in target_path and target in target_path:
                    for vertex in target_path[1:]:
                        vertex_path.append(vertex)
                else:
                    source_path_cost = calculate_cost(graph, source_path)
                    target_path_cost = calculate_cost(graph, target_path)
                    better_path = source_path if source_path_cost < target_path_cost else target_path
                    last_vertex = target if source_path_cost < target_path_cost else source

                    for vertex in better_path[1:]:
                        vertex_path.append(vertex)
                    vertex_path.append(last_vertex)
        else:
            vertex_path.append(source)
            vertex_path.append(target)
    return vertex_path

In [ ]:
graph.get_shortest_paths(v=0, to=2, weights="weight", output="vpath")

In [16]:
optimal_path = [9, 10, 9, 5, 6, 7, 6, 5, 9, 8, 4, 3, 16, 3, 4, 5, 10, 11, 15, 14, 0, 19, 0, 1, 2, 19, 2, 3, 16, 15, 17, 2, 19, 18, 17, 18, 19, 18, 14, 13, 14, 12, 13, 12, 11, 10, 9]

cost = 0
for index in range(len(optimal_path) - 1):
    source = optimal_path[index]
    target = optimal_path[(index + 1) % len(optimal_path)]
    cost += graph.es.find(_source=source, _target=target)['weight']

cost

272

In [28]:
def fitness2(graph: igraph.Graph, solution, solution_index):
    vertex_path = create_vertex_path(graph, solution)

    return -calculate_cost(graph, vertex_path)

In [21]:
population = list(map(lambda x: int(x), [9.0, 5.0, 4.0, 1.0, 0.0, 2.0, 22.0, 25.0, 29.0, 6.0, 3.0, 11.0, 14.0, 12.0, 13.0, 15.0, 17.0, 18.0, 24.0, 28.0, 27.0, 20.0, 19.0, 21.0, 23.0, 26.0, 8.0, 7.0, 16.0, 10.0]))

In [29]:
fitness2(graph, population, 0)

-312

In [80]:
vertex_optimal_path =  [0, 1, 2, 3, 4, 5, 4, 8, 9, 5, 6, 7, 6, 5, 10, 9, 10, 11, 12, 13, 12, 14, 0, 19, 2, 17, 15, 11, 15, 16, 3, 16, 15, 14, 13, 14, 18, 17, 18, 19, 0]
edge_df = graph.get_edge_dataframe()
my_edges = []
for index in range(len(vertex_optimal_path) - 1):
    begin = vertex_optimal_path[index]
    end = vertex_optimal_path[(index + 1) % len(vertex_optimal_path)]
    found = edge_df.query("(source == @begin & target == @end) | (source == @end & target == @begin)").head(1)
    edge = graph.es.find(_source=begin, _target=end).index
    my_edges.append(edge)
print(my_edges)

[0, 3, 5, 9, 11, 11, 10, 16, 14, 12, 13, 13, 12, 15, 17, 17, 18, 19, 21, 21, 22, 2, 1, 4, 6, 27, 20, 20, 26, 7, 7, 26, 24, 23, 23, 25, 28, 28, 29, 1]


In [81]:
len([0, 3, 5, 9, 11, 11, 10, 16, 14, 12, 13, 13, 12, 15, 17, 17, 18, 19, 21, 21, 22, 2, 1, 4, 6, 27, 20, 20, 26, 7, 7, 26, 24, 23, 23, 25, 28, 28, 29, 1])

40

In [86]:
line_graph: igraph.Graph = graph.linegraph()
for vertex in range(line_graph.vcount()):
    line_graph.vs[vertex]["weight"] = graph.es[vertex]["weight"]

_fitness(line_graph, my_edges, 0)

-206

In [95]:
def generate_random_permutations(len_permutation, num_permutations):
    population = [list(range(len_permutation)) for _ in range(num_permutations)]
    for solution in population:
        random.shuffle(solution)
    return population

numpy.array(generate_random_permutations(3, 4)).ndim

2

In [94]:
def generate_initial_population2(graph: igraph.Graph, population_size: int):
    population = [graph.get_edgelist() for _ in range(population_size)]
    chaos_population = []

    for solution in population:
        chaos_edge_list = []
        for a, b in solution:
            if random.random() > 0.5:
                chaos_edge_list.append((b, a))
            else:
                chaos_edge_list.append((a, b))
        random.shuffle(chaos_edge_list)
        chaos_population.append(chaos_edge_list)

    return chaos_population

cahos_population = generate_initial_population2(graph, 2)

numpy.array(cahos_population).ndim

3